# Change Composition Using MDAnalysis
We will use the following outline:
1. Import all necessary packages
2. Randomly select a specified number of DPPC
3. Edit those DPPC, preferably using MDAnalysis

In [47]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import MDAnalysis as mda

input_file = "dppc_bilayer.gro"
bilayer = mda.Universe(input_file)
bilayeratoms = bilayer.atoms
individualatoms = bilayeratoms.resids

/home/michael/anaconda2/lib/python2.7/site-packages/ipykernel/pylab/config.py:66: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  inline backend."""
/home/michael/anaconda2/lib/python2.7/site-packages/ipykernel/pylab/config.py:71: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  'retina', 'jpeg', 'svg', 'pdf'.""")
/home/michael/anaconda2/lib/python2.7/site-packages/ipykernel/pylab/config.py:85: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  use `figure_formats` instead)""")
/home/michael/anaconda2/lib/python2.7/site-packages/ipykernel/pylab/config.py:95: DeprecationWarning: metadata {'config': True} was set from the c

In [182]:
randomnumberselectiontest = np.random.randint(0,129,35)
randomnumberselectiontest = randomnumberselectiontest.tolist()
rlist = randomnumberselectiontest
sel = bilayer.select_atoms("resid 0")

for i in rlist:
    sel = sel + bilayer.select_atoms("resid %s" % i)

sel.resnames = "DBPC"
# At this point, sel is an atom selection with name DBPC, but they are all DPPC. We need to edit sel to remove later
# carbons

bilayer.select_atoms("name not NC3") #Figure out how to fix boolean here.

<AtomGroup with 420 atoms>

4/13/16:

We need to take sel and make a subselection that does not include atoms which are present in DBPC. Then, we need to make a subselection of the bilayer that doesn't include the original atoms in sel. The, we can merge these two subselections into an independent universe with 'merge' and export that as either pdb or gro. 

In [119]:
len(rlist)

35

In [79]:
selectiontest = bilayer.select_atoms("resname DPPC")

waterselectiontest = bilayer.select_atoms("resname W")

numberselectiontest = bilayer.select_atoms("resid 15")

randomnumberselectiontest = np.random.randint(0,101,35)
randomnumberselectiontest = randomnumberselectiontest.tolist()
